# Part 1: Merge LoRA Adapters

This notebook:
1. Loads saved LoRA parameters
2. Merges with base model
3. Saves merged model to Drive

**Requirements:** GPU enabled

**After this:** the model was saved in GGUF format via hugginface and downloaded using a separate notebook for test


In [ ]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
MODEL_DIR = "/content/drive/MyDrive/lab2_models"

Mounted at /content/drive


## Load Your Saved LoRA

In [ ]:
from unsloth import FastLanguageModel

print("="*80)
print("LOADING YOUR SAVED LORA")
print("="*80)

lora_path = os.path.join(MODEL_DIR, "optimized_lora_model")
print(f"\nLoading from: {lora_path}")
print("Loading in 16-bit for better GGUF quality...\n")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=lora_path,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=False,
)

print("✓ LoRA loaded successfully")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
LOADING YOUR SAVED LORA

Loading from: /content/drive/MyDrive/lab2_models/optimized_lora_model
Loading in 16-bit for better GGUF quality...

==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

Unsloth 2025.11.4 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


✓ LoRA loaded successfully


## Merge and Save to Google Drive

In [ ]:
print("\n" + "="*80)
print("MERGING AND SAVING TO GOOGLE DRIVE")
print("="*80)

merged_path = "/content/drive/MyDrive/lab2_models/merged_optimized_for_gguf"

print(f"\nMerging and saving to: {merged_path}")
print("This takes 3-5 minutes...\n")

# Merge and save
model.save_pretrained_merged(
    merged_path,
    tokenizer,
    save_method="merged_16bit",
    safe_serialization=True,
    max_shard_size="2GB",
)

print("\n✓ Model merged and saved")

# Save tokenizer
print("\nSaving tokenizer...")
tokenizer.save_pretrained(merged_path, legacy_format=False)
# Save tokenizer
print("\nSaving tokenizer...")
tokenizer.save_pretrained(merged_path, legacy_format=False)

# Copy config from base Llama 3.2 1B (guarantees it works)
print("\nCopying config from base model cache...")
import shutil
import glob

# Find base model config in HuggingFace cache
base_config_pattern = "/root/.cache/huggingface/hub/models--unsloth--Llama-3.2-1B/snapshots/*/config.json"
configs = glob.glob(base_config_pattern)

if configs:
    # Copy the working config
    shutil.copy(configs[0], os.path.join(merged_path, "config.json"))
    print(f"✓ Copied working config from: {configs[0]}")

    # Verify it worked
    import json
    with open(os.path.join(merged_path, "config.json")) as f:
        config = json.load(f)
    print(f"  model_type: {config.get('model_type')}")
    print(f"  architectures: {config.get('architectures')}")
else:
    print("✗ Base model config not found in cache")
    print("Using current config (may have issues)")

print("\n✓ Config setup complete")




MERGING AND SAVING TO GOOGLE DRIVE

Merging and saving to: /content/drive/MyDrive/lab2_models/merged_optimized_for_gguf
This takes 3-5 minutes...

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...


Unsloth: Copying 1 files from cache to `/content/drive/MyDrive/lab2_models/merged_optimized_for_gguf`: 100%|██████████| 1/1 [00:56<00:00, 56.64s/it]


Successfully copied all 1 files from cache to `/content/drive/MyDrive/lab2_models/merged_optimized_for_gguf`
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:49<00:00, 49.26s/it]


Unsloth: Merge process complete. Saved to `/content/drive/MyDrive/lab2_models/merged_optimized_for_gguf`

✓ Model merged and saved

Saving tokenizer...

Saving tokenizer...

Copying config from base model cache...
✗ Base model config not found in cache
Using current config (may have issues)

✓ Config setup complete


## Verify Files

In [ ]:
# Ensure model_type is at the top of config (some parsers need this)
print("\nReordering config with model_type first...")
import json
from collections import OrderedDict

config_path = os.path.join(merged_path, "config.json")
with open(config_path, 'r') as f:
    config_dict = json.load(f)

# Create ordered dict with critical keys first
ordered_config = OrderedDict()
ordered_config['model_type'] = 'llama'
ordered_config['architectures'] = ['LlamaForCausalLM']
ordered_config['vocab_size'] = 128256

# Add rest of keys
for k, v in config_dict.items():
    if k not in ['model_type', 'architectures', 'vocab_size']:
        ordered_config[k] = v

# Save with critical keys at top
with open(config_path, 'w') as f:
    json.dump(ordered_config, f, indent=2)

print("✓ Config reordered with model_type at top")


Reordering config with model_type first...
✓ Config reordered with model_type at top


In [ ]:
print("="*80)
print("VERIFICATION")
print("="*80)

merged_path = "/content/drive/MyDrive/lab2_models/merged_optimized_for_gguf"

required_files = [
    "config.json",
    "tokenizer.json",
    "tokenizer_config.json",
    "special_tokens_map.json"
]

print(f"\nChecking {merged_path}:\n")

all_good = True
for f in required_files:
    fpath = os.path.join(merged_path, f)
    if os.path.exists(fpath):
        print(f"✓ {f}")
    else:
        print(f"✗ {f} MISSING!")
        all_good = False

# Check for model files
import glob
model_files = glob.glob(os.path.join(merged_path, "*.safetensors"))
if model_files:
    print(f"\n✓ Found {len(model_files)} model shard(s)")
    total_size = sum(os.path.getsize(f) for f in model_files) / (1024**3)
    print(f"  Total size: {total_size:.2f} GB")
else:
    print("\n✗ No model files found!")
    all_good = False

print("\n" + "="*80)
if all_good:
    print("SUCCESS! Merged model ready for GGUF conversion")
    print("\nNext step: Run Part 2 notebook to convert to GGUF")
else:
    print("ERROR: Some files are missing!")
print("="*80)

VERIFICATION

Checking /content/drive/MyDrive/lab2_models/merged_optimized_for_gguf:

✓ config.json
✓ tokenizer.json
✓ tokenizer_config.json
✓ special_tokens_map.json

✓ Found 1 model shard(s)
  Total size: 2.30 GB

SUCCESS! Merged model ready for GGUF conversion

Next step: Run Part 2 notebook to convert to GGUF


In [ ]:
print("\n" + "="*80)
print("UPLOADING TO HUGGINGFACE")
print("="*80)

# Install huggingface_hub
!pip install -q huggingface_hub

from huggingface_hub import HfApi, login
from getpass import getpass

# Login
print("\n1. Go to: https://huggingface.co/settings/tokens")
print("2. Create a token with WRITE permissions")
print("3. Paste it below:\n")

token = "not needed"
login(token=token)

# Get username
hf_username = "lippa6602"

# Create repo name
repo_name = f"{hf_username}/llama-3.2-1b-finetome-optimized"

print(f"\nUploading to: {repo_name}")
print("This takes 5-10 minutes (uploading ~2.4 GB)...\n")

api = HfApi()

# Create repo (private by default)
try:
    api.create_repo(
        repo_id=repo_name,
        exist_ok=True,
        private=True,
        repo_type="model"
    )
    print(f"✓ Repo created/exists")
except Exception as e:
    print(f"Repo creation: {e}")

# Upload all files from merged model
api.upload_folder(
    folder_path=merged_path,
    repo_id=repo_name,
    repo_type="model",
    commit_message="Upload fine-tuned Llama 3.2 1B"
)

print("\n" + "="*80)
print("✅ UPLOAD COMPLETE!")
print("="*80)
print(f"\nModel URL: https://huggingface.co/{repo_name}")
print("\n📝 NEXT STEPS:")
print("1. Go to the URL above")
print("2. Wait 1-2 minutes for HF to process the model")
print("3. Go to 'Files and versions' tab")
print("4. Click the 3 dots next to any file → 'Create GGUF'")
print("5. Select 'Q4_K_M' quantization")
print("6. Download the generated GGUF file")
print("\nOR use this command to download later:")
print(f"  huggingface-cli download {repo_name} --include '*.gguf' --local-dir ./model")


UPLOADING TO HUGGINGFACE

1. Go to: https://huggingface.co/settings/tokens
2. Create a token with WRITE permissions
3. Paste it below:


Uploading to: lippa6602/llama-3.2-1b-finetome-optimized
This takes 5-10 minutes (uploading ~2.4 GB)...

✓ Repo created/exists


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...d_for_gguf/tokenizer.json:  97%|#########7| 16.7MB / 17.2MB            

  ...or_gguf/model.safetensors:   1%|          | 16.7MB / 2.47GB            


✅ UPLOAD COMPLETE!

Model URL: https://huggingface.co/lippa6602/llama-3.2-1b-finetome-optimized

📝 NEXT STEPS:
1. Go to the URL above
2. Wait 1-2 minutes for HF to process the model
3. Go to 'Files and versions' tab
4. Click the 3 dots next to any file → 'Create GGUF'
5. Select 'Q4_K_M' quantization
6. Download the generated GGUF file

OR use this command to download later:
  huggingface-cli download lippa6602/llama-3.2-1b-finetome-optimized --include '*.gguf' --local-dir ./model


## Summary

✅ **Merged model saved to:** `/content/drive/MyDrive/lab2_models/merged_optimized_for_gguf/`

**Next:** Run the "Part 2: Convert to GGUF" notebook to finish the conversion!